In [1]:
!pip install beir
!pip install elasticsearch
!pip install -U sentence-transformers
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/21

In [2]:
from beir import util
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.search.lexical import BM25Search as BM25
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from beir.retrieval import models

import pathlib, os, string
from tqdm.notebook import tqdm

import spacy

import time
import numpy as np
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp
import string

from sentence_transformers import SentenceTransformer

import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Application running on {device}")

/usr/local/lib/python3.9/dist-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Application running on cpu


## Download and setup the Elasticsearch instance

In [3]:
%%bash

wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.9.2/
shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512 

elasticsearch-oss-7.9.2-linux-x86_64.tar.gz: OK


In [4]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

In [5]:
for i in tqdm(range(20), desc = 'Let the Elasticsearch instance start'): time.sleep(1)

Let the Elasticsearch instance start:   0%|          | 0/20 [00:00<?, ?it/s]

In [6]:
%%bash

ps -ef | grep elasticsearch

root         962     959  0 20:44 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch
daemon       963     962 78 20:44 ?        00:00:15 /content/elasticsearch-7.9.2/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/elasticsearch-942279233417357604 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:filecoun

In [7]:
%%bash

curl -sX GET "localhost:9200/"

{
  "name" : "7b7a77676715",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "gNNe7pJ8QVOsZRPA4lq5Fg",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


# Available Datasets

| Dataset   | Website| BEIR-Name | Domain     | Relevancy| Queries  | Documents | Avg. Docs/Q | Download | 
| -------- | -----| ---------| ----------- | ---------| ---------| --------- | ------| ------------| 
| MSMARCO    | [``Homepage``](https://microsoft.github.io/msmarco/)| ``msmarco`` | Misc.       |  Binary  |  6,980   |  8.84M     |    1.1 | Yes |  
| TREC-COVID |  [``Homepage``](https://ir.nist.gov/covidSubmit/index.html)| ``trec-covid``| Bio-Medical |  3-level|50|  171K| 493.5 | Yes | 
| NFCorpus   | [``Homepage``](https://www.cl.uni-heidelberg.de/statnlpgroup/nfcorpus/) | ``nfcorpus``  | Bio-Medical |  3-level |  323     |  3.6K     |  38.2 | Yes |
| BioASQ     | [``Homepage``](http://bioasq.org) | ``bioasq``| Bio-Medical |  Binary  |   500    |  14.91M    |  8.05 | No | 
| NQ         | [``Homepage``](https://ai.google.com/research/NaturalQuestions) | ``nq``| Wikipedia   |  Binary  |  3,452   |  2.68M  |  1.2 | Yes | 
| HotpotQA   | [``Homepage``](https://hotpotqa.github.io) | ``hotpotqa``| Wikipedia   |  Binary  |  7,405   |  5.23M  |  2.0 | Yes |
| FiQA-2018  | [``Homepage``](https://sites.google.com/view/fiqa/) | ``fiqa``    | Finance     |  Binary  |  648     |  57K    |  2.6 | Yes | 
| Signal-1M (RT) | [``Homepage``](https://research.signal-ai.com/datasets/signal1m-tweetir.html)| ``signal1m`` | Twitter     |  3-level  |   97   |  2.86M  |  19.6 | No |
| TREC-NEWS  | [``Homepage``](https://trec.nist.gov/data/news2019.html) | ``trec-news``    | News     |  5-level  |   57    |  595K    |  19.6 | No |
| ArguAna    | [``Homepage``](http://argumentation.bplaced.net/arguana/data) | ``arguana`` | Misc.       |  Binary  |  1,406     |  8.67K    |  1.0 | Yes |
| Touche-2020| [``Homepage``](https://webis.de/events/touche-20/shared-task-1.html) | ``webis-touche2020``| Misc.       |  6-level  |  49     |  382K    |  49.2 |  Yes |
| CQADupstack| [``Homepage``](http://nlp.cis.unimelb.edu.au/resources/cqadupstack/) | ``cqadupstack``| StackEx.      |  Binary  |  13,145 |  457K  |  1.4 |  Yes |
| Quora| [``Homepage``](https://www.quora.com/q/quoradata/First-Quora-Dataset-Release-Question-Pairs) | ``quora``| Quora  | Binary  |  10,000     |  523K    |  1.6 |  Yes | 
| DBPedia | [``Homepage``](https://github.com/iai-group/DBpedia-Entity/) | ``dbpedia-entity``| Wikipedia |  3-level  |  400    |  4.63M    |  38.2 |  Yes | 
| SCIDOCS| [``Homepage``](https://allenai.org/data/scidocs) | ``scidocs``| Scientific |  Binary  |  1,000     |  25K    |  4.9 |  Yes | 
| FEVER| [``Homepage``](http://fever.ai) | ``fever``| Wikipedia     |  Binary  |  6,666     |  5.42M    |  1.2|  Yes | 
| Climate-FEVER| [``Homepage``](http://climatefever.ai) | ``climate-fever``| Wikipedia |  Binary  |  1,535     |  5.42M |  3.0 |  Yes |
| SciFact| [``Homepage``](https://github.com/allenai/scifact) | ``scifact``| Scientific |  Binary  |  300     |  5K    |  1.1 |  Yes |


# Data Loading

In [8]:
def download_dataset(dataset):
  data_path = f'datasets/{dataset}'
  url = f'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{dataset}.zip'
  out_dir = os.path.join(os.getcwd(), 'datasets')
  data_path = util.download_and_unzip(url, out_dir)
  print(f'Dataset downloaded here: {data_path}')
  return GenericDataLoader(data_path).load(split="test")

            #light        medium      heavy
datasets = ['scifact', 'trec-covid', 'fever']
datasets_data = {}

for dataset in datasets:
  corpus, queries, qrels = download_dataset(dataset)
  datasets_data[dataset] = {
      'corpus': corpus,
      'queries': queries,
      'qrels': qrels
  }

/content/datasets/scifact.zip:   0%|          | 0.00/2.69M [00:00<?, ?iB/s]

Dataset downloaded here: /content/datasets/scifact


  0%|          | 0/5183 [00:00<?, ?it/s]

/content/datasets/trec-covid.zip:   0%|          | 0.00/70.5M [00:00<?, ?iB/s]

Dataset downloaded here: /content/datasets/trec-covid


  0%|          | 0/171332 [00:00<?, ?it/s]

/content/datasets/fever.zip:   0%|          | 0.00/1.15G [00:00<?, ?iB/s]

Dataset downloaded here: /content/datasets/fever


  0%|          | 0/5416568 [00:00<?, ?it/s]

In [9]:
def embeddings(model, corpus, queries, qrels):
  retriever = EvaluateRetrieval(model, score_function='dot')

  #### Retrieve dense results (format of results is identical to qrels)
  results = retriever.retrieve(corpus, queries)

  ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)
  return results, retriever.k_values, {'ndcg': ndcg, 'map': _map, 'recall': recall, 'precision': precision}

In [10]:
def print_res(score_dict):
  for score, res in score_dict.items():
    print(score)
    for k, r in res.items():
      print(f'\t{k}\t->\t{r}')
    print('\n')

# Sparse Embeddings with BM25

In [11]:
nlp = spacy.load('en_core_web_sm')
stopwords = nlp.Defaults.stop_words
clean_tokens = lambda tokens : ' '.join([token.lemma_.lower() if token not in stopwords and not token.is_punct else '' for token in tokens])

In [ ]:
def pre_process(elem_to_preprocess):
  key, val = elem_to_preprocess
  if type(val) is dict:
    return key, {
        'title': clean_tokens(nlp(val['title'])),
        'text': clean_tokens(nlp(val['text']))
    }
  else: return key, clean_tokens(nlp(val))

def query_documents_preprocessing(dataset_name, documents, queries):
  new_queries = {}
  new_documents = {}

  for text, iter, res in zip(('Documents', 'Queries'), (documents, queries), (new_documents, new_queries)):
    with ProcessPoolExecutor(max_workers=mp.cpu_count()) as executor:
      for id, query_doc in list(tqdm(executor.map(pre_process, iter.items()),
                                     total=len(iter.items()), desc=f'{dataset_name} - {text} Pre-Processing')):
        res[id] = query_doc

  return new_documents, new_queries


pre_processed_data = {}
for dataset, values in datasets_data.items(): 
  pre_proc_corpus, pre_proc_queries = query_documents_preprocessing(dataset, values['corpus'], values['queries'])
  pre_processed_data[dataset] = {
      'pre_processed_corpus': pre_proc_corpus,
      'pre_processed_queries': pre_proc_queries
  }

In [ ]:
def sparse_embeddings_bm25(dataset_name, corpus, queries):
  hostname = 'localhost' 
  index_name = 'scifact' 
  initialize = True # True, will delete existing index with same name and reindex all documents

  print(f'{dataset_name} - BM25')
  model = BM25(index_name=index_name, hostname=hostname, initialize=initialize)
  return embeddings(model, corpus, queries, qrels)

sparse_datasets_embeddings = {}
for dataset, pre_pro_data in pre_processed_data.items():
  sparse_vector, sparse_k_values, sparse_score_dict = sparse_embeddings_bm25(dataset, pre_pro_data['pre_processed_corpus'],
                                                                             pre_pro_data['pre_processed_queries'])
  print(f'\nSparse retrieved evaluation for k in: {sparse_k_values}')
  print_res(sparse_score_dict)
  sparse_datasets_embeddings[dataset] = {
      'sparse_vector': sparse_vector,
      'sparse_k_values': sparse_k_values,
      'sparse_score_dict': sparse_score_dict
  }

# Dense Embeddings with SentenceBert and all-MiniLM-L6-v2

In [ ]:
def dense_embeddings_sbert(dataset_name, corpus, queries):
  model = DRES(models.SentenceBERT('all-MiniLM-L6-v2'), batch_size=16)
  print(f'{dataset_name} - SBERT')
  return embeddings(model, corpus, queries, qrels)

dense_datasets_embeddings = {}
for dataset, pre_pro_data in pre_processed_data.items():
  dense_vector, dense_k_values, dense_score_dict = dense_embeddings_sbert(dataset, corpus, queries)
  print(f'\nDense etrieved evaluation for k in: {dense_k_values}')
  print_res(dense_score_dict)
  dense_datasets_embeddings[dataset] = {
      'dense_vector': dense_vector,
      'dense_k_values': dense_k_values,
      'dense_score_dict': dense_score_dict
  }

# Ground truth at k

In [ ]:
def ground_truth_merging(sparse_vector, dense_vector, top_k=[1, 3, 5, 10, 100, 1000]):
  ground_truth = {}
  for k in top_k:

    ground_truth[k] = {}

    for (query, sparse_dic), (_, dense_dic) in tqdm(zip(sparse_vector.items(), dense_vector.items()), total=len(sparse_vector.items()), desc=f'Retriving the ground truth for the top {k}'):
      
      ground_truth[k][query] = {}

      sorted_sparse_score = dict(sorted(sparse_dic.items(), key=lambda item: item[1], reverse=True)[0:k])
      sorted_dense_score = dict(sorted(dense_dic.items(), key=lambda item: item[1], reverse=True)[0:k])

      ground_truth[k][query] = {doc_id: 1 for doc_id in set(sorted_sparse_score) | set(sorted_dense_score)}
  
  return ground_truth

datasets_ground_truth_k = {}
for (dataset, sparse_elem), (_, dense_elem) in zip(sparse_datasets_embeddings.items(), dense_datasets_embeddings.items()):
  datasets_ground_truth_k[dataset] = ground_truth_merging(sparse_elem['sparse_vector'], dense_elem['dense_vector'])

# Merging Dense and Sparse Embeddings

In [ ]:
def merging_top_k(sparse_vector, dense_vector, top_k=[1, 3, 5, 10, 100, 1000]):
  merged_vector = {}
  k_values = [1, 3, 5, 10, 100, 1000]

  for (query, sparse_dic), (_, dense_dic) in tqdm(zip(sparse_vector.items(), dense_vector.items()), total=len(sparse_vector.items()), desc='Obtaining the merged vector'):
    merged_vector[query] = {doc_id: sparse_dic.get(doc_id, 0) + dense_dic.get(doc_id, 0) for doc_id in set(sparse_dic) | set(dense_dic)}
  
  return merged_vector

datasets_merged_top_k_vector = {}
for (dataset, sparse_elem), (_, dense_elem) in zip(sparse_datasets_embeddings.items(), dense_datasets_embeddings.items()):
  datasets_merged_top_k_vector[dataset] = merging_top_k(sparse_elem['sparse_vector'], dense_elem['dense_vector'])

# Evaluating the top k respect the ground truth

In [ ]:
def evaluate_top_k(merged_top_k_vector, ground_truth_k):
  k_values = [1, 3, 5, 10, 100, 1000]

  evaluate_top_k_results = {}

  for k, ground_truth in tqdm(ground_truth_k.items(), total=len(ground_truth_k.items()), desc='Evaluating the retreived top K among the sparse and dense embeddings'):
    ndcg, _map, recall, precision = EvaluateRetrieval.evaluate(ground_truth, merged_top_k_vector, k_values=k_values)
    evaluate_top_k_results[k] = {'ndcg': ndcg, 'map': _map, 'recall': recall, 'precision': precision}
  
  return evaluate_top_k_results

datasets_results_evaluate_top_k = {}
for (dataset, merged_elem), (_, g_truth_elem) in zip(datasets_merged_top_k_vector.items(), datasets_ground_truth_k.items()):
  datasets_results_evaluate_top_k[dataset] = evaluate_top_k(merged_elem, g_truth_elem)

In [ ]:
for dataset, dataset_result in datasets_results_evaluate_top_k.items():
  print(f'--------------------- {dataset} ---------------------')
  for k, result in dataset_result.items():
    print(f'---- TOP {k} results ----')
    print_res(result)
    print('\n')